In [3]:
from dotenv import load_dotenv
import os

def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")
        
        
# Load environment variables from the .env file
load_dotenv("./API_keys.env")

# Access the environment variables
os.getenv('OPENAI_API_KEY')
_set_if_undefined("OPENAI_API_KEY")


os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"

In [4]:
from langchain_community.document_loaders import TextLoader, WebBaseLoader

# Getting data for QA.
loader = TextLoader("./data_source.txt")

docs = loader.load()

docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': './data_source.txt'}, page_content='ข้อมูลโรงเรียนสมมติ\n\nชื่อโรงเรียน: โรงเรียนประชาบำรุงศึกษา\n\nที่ตั้ง:\n123/45 ถนนการศึกษา แขวงการเรียนรู้ เขตการศึกษา กรุงเทพฯ 10110\n\nโทรศัพท์: 02-123-4567\n\nเว็บไซต์: www.prachabamrung.ac.th\n\nการรับสมัครนักเรียน\n\n\t•\tวันเปิดรับสมัคร: 1 กุมภาพันธ์ 2567\n\t•\tวันปิดรับสมัคร: 15 มีนาคม 2567\n\t•\tวิธีการสมัคร: สมัครผ่านเว็บไซต์ของโรงเรียน หรือสมัครด้วยตนเองที่โรงเรียน\n\t•\tเอกสารที่ต้องใช้:\n\t•\tสำเนาทะเบียนบ้าน\n\t•\tสำเนาบัตรประชาชนผู้ปกครอง\n\t•\tสำเนาใบเกิดของนักเรียน\n\t•\tรูปถ่ายขนาด 1 นิ้ว จำนวน 2 รูป\n\t•\tค่าเล่าเรียนสำหรับปีการศึกษา 2567:\n\t•\tอนุบาล 1-3: 30,000 บาท/ปี\n\t•\tประถมศึกษาปีที่ 1-6: 40,000 บาท/ปี\n\t•\tมัธยมศึกษาปีที่ 1-6: 50,000 บาท/ปี\n\t•\tทุนการศึกษา: โรงเรียนมีทุนการศึกษาสำหรับนักเรียนที่มีผลการเรียนดีและมีความสามารถพิเศษ\n\nหลักสูตรและโปรแกรมการเรียนการสอน\n\n\t•\tระดับอนุบาล:\n\t•\tอนุบาล 1: เรียนรู้พื้นฐานการอ่านและเขียน, การสร้างความคิดสร้างสรรค์ผ่านกิจกรรมศิลปะ\n\t•\tอนุบาล 2-

In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [23]:
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Split text into chunks separated.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

# Text Vectorization.
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_messages(
        [
            (
                "assistant",
                """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. Answer in the language that users use. Your are a male.
                Question: {question} 
                Context: {context} 
                Answer:"""
                
            ),
            # MessagesPlaceholder(variable_name="messages"),
            # MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [35]:
import random
from math import inf

def QA_sample_test(chain, quesion_test:list[str], num_samples:int=inf):
    
    # Ensure there are at least 5 lines in the list
    num_samples = min(10, len(quesion_test))
    sample_quesion = random.sample(quesion_test, num_samples)
        
    resaults = []
    for quesion in sample_quesion:
        answer = chain.invoke(quesion)
        resaults.append({'quesion': quesion, 'answer': answer})   
        
    return resaults 

In [38]:
with open('./user_question_test_sets.txt', 'r') as file:
    quesion_test = file.readlines()    

resaults = QA_sample_test(rag_chain, quesion_test, num_samples=30)

Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')


In [39]:
resaults

[{'quesion': 'โรงเรียนนี้มีชมรมอะไรบ้าง?\n',
  'answer': 'โรงเรียนนี้มีชมรมคอมพิวเตอร์, ชมรมดนตรีไทย, ชมรมภาษาอังกฤษ, ชมรมว่ายน้ำ, และชมรมหุ่นยนต์ นอกจากนี้ยังมีกิจกรรมกีฬา เช่น ฟุตบอล, แบดมินตัน, และบาสเก็ตบอลด้วยครับ.'},
 {'quesion': 'งานเปิดบ้านวิชาการจะจัดขึ้นเมื่อไหร่?\n',
  'answer': 'งานเปิดบ้านวิชาการจะจัดขึ้นระหว่างวันที่ 5-7 กุมภาพันธ์ 2568 ซึ่งจะมีการแสดงผลงานทางวิชาการของนักเรียนทุกระดับชั้น.'},
 {'quesion': 'ภาคเรียนที่ 1 จะเริ่มเมื่อไหร่?\n',
  'answer': 'ภาคเรียนที่ 1 จะเริ่มในวันที่ 16 พฤษภาคม 2567 โดยจะมีพิธีเปิดภาคเรียนและการแสดงของนักเรียนในวันนั้นด้วยค่ะ'},
 {'quesion': 'โรงเรียนมีสระว่ายน้ำไหม?\n',
  'answer': 'โรงเรียนมีสระว่ายน้ำมาตรฐานโอลิมปิก พร้อมทีมผู้ฝึกสอนมืออาชีพ และมีการสอนว่ายน้ำสำหรับนักเรียนทุกระดับด้วยครับ'},
 {'quesion': 'อาหารที่โรงอาหารมีเมนูสุขภาพหรือไม่?\n',
  'answer': 'โรงอาหารมีเมนูสุขภาพสำหรับนักเรียนที่ต้องการอาหารพิเศษด้วยครับ ดังนั้นคุณสามารถเลือกเมนูสุขภาพได้ตามต้องการครับ'},
 {'quesion': 'กิจกรรมกีฬาสีจัดขึ้นวันไหน?\n',
  'answer': 'ไม่ท

In [42]:
ans = rag_chain.invoke("เว็บโรงเรียน")
ans

Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')


'เว็บโรงเรียนที่คุณสนใจคือ โรงเรียนประชาบำรุงศึกษา ที่ตั้งอยู่ที่ 123/45 ถนนการศึกษา แขวงการเรียนรู้ เขตการศึกษา กรุงเทพฯ 10110 โทรศัพท์: 02-123-4567 คุณสามารถเข้าไปดูรายละเอียดเพิ่มเติมได้ที่เว็บไซต์ www.prachabamrung.ac.th'

In [40]:
#save resaults
with open('QA_test.txt', 'w') as file:
    # Write each item in the list to a new line in the file
    for item in resaults:
        file.write(f"{item}\n")